In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import statsmodels.api as sm
import matplotlib.pyplot as plt

import yfinance as yf

In [11]:
df_ff = pd.read_csv('/Users/marius/desktop/quant_research_project/data/fama_french_factors.csv', index_col=0, parse_dates=True)
df_ff = df_ff[df_ff.index >= '2015-01-01']
df_ff.head()


,Mkt-RF,SMB,HML,RF
Date,,,,
2015-01-01,-2.87,-1.13,-0.64,0.02
2015-02-01,5.75,2.22,1.27,0.02
2015-03-01,-1.54,-0.61,-0.35,0.02
2015-04-01,0.42,0.17,0.10,0.02
2015-05-01,1.33,0.52,0.30,0.02


In [12]:
start_date = datetime(2015, 1, 1)
end_date = datetime.today()
sectors = ['XLB', 'XLC', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY', 'XLE']

df_sectors = pd.DataFrame()
for sym in sectors:
    print(sym)
    # df = downloadpdr.DataReader(name=sym, data_source='yahoo', start=start_date, end=end_date)
    df = yf.download(sym, start=start_date, end=end_date)
    df = df[['Close']]
    df.columns = [sym]
    df_sectors = pd.concat([df_sectors, df], axis=1, join='outer')

XLB


[*********************100%***********************]  1 of 1 completed


XLC


[*********************100%***********************]  1 of 1 completed


XLF


...

[*********************100%***********************]  1 of 1 completed


XLY


[*********************100%***********************]  1 of 1 completed


XLE


[*********************100%***********************]  1 of 1 completed


In [17]:
df_sec_ret = df_sectors.resample('ME').last()
df_sec_ret.index = df_sec_ret.index.to_period()
df_sec_ret = df_sec_ret.pct_change()
df_sec_ret = df_sec_ret.dropna()
df_sec_ret.head()

,XLB,XLC,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,XLE
Date,,,,,,,,,,,
2018-07,0.028586,-0.021603,0.051147,0.073852,0.020872,0.039589,0.010394,0.015974,0.065541,0.018024,0.015539
2018-08,-0.007701,0.014858,0.013596,0.002340,0.065990,0.003920,0.024205,0.012881,0.043292,0.051047,-0.034751
2018-09,-0.017954,-0.000963,-0.022155,0.021774,-0.000192,0.009872,-0.026461,-0.006484,0.029542,0.005304,0.024460
2018-10,-0.091835,-0.060408,-0.047136,-0.108673,-0.080047,0.020026,-0.015946,0.019753,-0.067788,-0.101006,-0.113282
2018-11,0.038016,-0.022372,0.026256,0.038065,-0.019625,0.022723,0.054846,0.035388,0.080834,0.024767,-0.015635


In [28]:
df_sec_ret = df_sectors.resample('ME').last()
df_sec_ret = df_sec_ret.pct_change()
df_sec_ret = df_sec_ret.dropna()

df_sec_ret.index = df_sec_ret.index.to_period('M')

common_index = df_sec_ret.index.intersection(df_ff.index)
df_sec_ret = df_sec_ret.loc[common_index]
df_ff_aligned = df_ff.loc[common_index]

df_sec_ret = df_sec_ret.apply(lambda x: x - df_ff_aligned['RF']/100.0)
df_sec_ret.dropna(axis=0, inplace=True)

df_Y = df_sec_ret
df_X = df_ff_aligned[['Mkt-RF', 'SMB', 'HML']]/100.0
df_X = df_X.loc[df_Y.index]

print(f'{df_Y.shape[1]} stocks, {df_X.shape[1]} factors, {df_Y.shape[0]} time steps')

df_X = sm.add_constant(df_X, prepend=False)

11 stocks, 3 factors, 78 time steps


In [31]:
# fama_macbeth step one: time-series regression ==> factor exposures
beta = pd.DataFrame()             # factor exposures
for sym in df_Y.columns:
    model = sm.OLS(df_Y[sym], df_X)
    results = model.fit()
    beta = pd.concat([beta, pd.DataFrame([results.params[:3]], index=[sym])])
beta

,Mkt-RF,SMB,HML
XLB,8.281941,42.623783,-107.521450
XLC,4.901673,-134.269303,217.540416
XLF,16.546327,-32.060896,-14.082398
XLI,3.021062,25.558898,-53.494126
XLK,-11.812420,19.299585,24.850457
XLP,-0.213919,22.954815,-36.635269
XLRE,-4.303253,190.758596,-310.696137
XLU,-5.617232,52.239840,-63.697204
XLV,-6.674238,53.086421,-59.629907
XLY,13.732839,-12.014531,-35.843955


In [ ]:
#It makes sense that utilities (XLU) has the lowest market premium beta (-5.62), as it's a defensive sector. 
#Due to the historical oil price collapse, it is not surprising to see energy (XLE) with an extremely high market beta (119.92). 
#XLE also has a large negative exposure to the HML factor (-66.31), which is counterintuitive given that energy companies are typically value stocks with high book-to-market ratios. 
#This negative HML beta might reflect that during the oil collapse period (2015-2016), traditional value metrics became less relevant, or that the sector behaved more like distressed growth stocks.